In [ ]:
REQUEST_ID = 0
AOI = 'POLYGON ((-94.783452 42.110879, -94.77976099999999 42.110799, -94.779718 42.117867, -94.78068399999999 42.117898, -94.78089799999999 42.117676, -94.781499 42.117691, -94.78179900000001 42.117755, -94.781756 42.118026, -94.783258 42.118073, -94.783452 42.117978, -94.783452 42.115129, -94.783452 42.110879))'
START_DATE = '2020-07-07'
END_DATE = '2020-07-14'

In [ ]:
import os
import shutil
import json
import time
import cv2
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import shapely

from tqdm.notebook import tqdm
from os.path import join, basename, split
from scipy.ndimage import rotate
from rasterio import features
from rasterio.merge import merge
from rasterio.warp import (aligned_target,
                           calculate_default_transform,
                           reproject,
                           Resampling)
from shapely.geometry import Polygon, shape, LinearRing, box
import shapely.wkt
from pathlib import Path
from datetime import datetime
import matplotlib.pylab as plt
import seaborn as sns

import rasterio.mask as riomask

from code.download.utils import get_tiles, _check_folder, check_nodata, get_min_clouds, transform_resolution
from code.download.load_tiles import load_images
from code.index_research import combine_bands, calculate_ndvi, calculate_tcari, calculate_msr_g
from code.plant_stress import PlantStress
from code.utils import stitch_tiles

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [ ]:
NB_USER = os.getenv('NB_USER')
BASE = f"/home/{NB_USER}/work"
DATA_DIR = f"/home/{NB_USER}/work"

API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
sentinel_tiles_path = os.path.join(BASE, "notebooks/pbdnn/sentinel2grid.geojson")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, "results/dd")
PBD_DIR = os.path.join(BASE, "notebooks/pbdnn")

BANDS = {'B04','B08'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

In [ ]:
b04_tiles, b08_tiles = [], []
date_tile_info = get_tiles(aoi_filename, sentinel_tiles_path)

In [ ]:
loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)
needed_num_tiles = len(checked.keys())

while len(b04_tiles) < needed_num_tiles and len(b08_tiles) < needed_num_tiles:

    for i, tile in date_tile_info.iterrows():
        if len(checked[tile.tileID]) < 1:
            continue
                
        tile_folder = Path(sorted(checked[tile.tileID])[0])
        tile_files = list(tile_folder.rglob('*.jp2'))
        b04_tiles.append([x for x in tile_files if '_B04_' in str(x)][0])
        b08_tiles.append([x for x in tile_files if '_B08_' in str(x)][0])
        print(f'filtered: {tile_folder}')
        
    CONSTRAINTS['NODATA_PIXEL_PERCENTAGE'] += 5
    CONSTRAINTS['CLOUDY_PIXEL_PERCENTAGE'] += 5
    loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
    checked = check_nodata(loadings, PRODUCT_TYPE)
    if CONSTRAINTS['NODATA_PIXEL_PERCENTAGE'] > 70:
        raise Exception(f'No clean raster found for period from {START_DATE} to {END_DATE}, skipping, {e}')

In [ ]:
if len(b04_tiles) > 1:
    b04_tile = stitch_tiles(b04_tiles, str(b04_tiles[0]).replace('.jp2', '_merged.tif'))
    b08_tile = stitch_tiles(b08_tiles, str(b08_tiles[0]).replace('.jp2', '_merged.tif'))
else:
    b04_tile = str(b04_tiles[0])
    b08_tile = str(b08_tiles[0])

In [ ]:
os.makedirs(f'{RESULTS_DIR}/plant_stress/{REQUEST_ID}', exist_ok=True)
ndvi_path = os.path.join(RESULTS_DIR, f'plant_stress/{REQUEST_ID}/{REQUEST_ID}_ndvi.tif')
if not os.path.exists(ndvi_path):
    calculate_ndvi(b04_tile, b08_tile, out_path=ndvi_path, nodata=np.nan)

In [ ]:
min_ndvi = 0.3
z_score = 5
z_score_anom = 1
colors = {"Normal Growth": (0, 0, 0), "Anomaly": (182, 10, 28)}

field = gpd.read_file(aoi_filename)

In [ ]:
ps = PlantStress(min_ndvi=min_ndvi, noise_z_score=z_score, anomaly_z_score=z_score_anom)

In [ ]:
ps.segment_field(field, ndvi_path, START_DATE, END_DATE, REQUEST_ID)

In [ ]:
remove_aux_rasters = True

if remove_aux_rasters:
    
    try:
        shutil.rmtree(tile_folder)
    except FileNotFoundError:
        print(f'Error removing directory {tile_folder}')
        
    try:
        os.remove(ndvi_path)
    except FileNotFoundError:
        print(f'File {ndvi_path} not found')
        
    try:
        os.remove(aoi_filename)
    except FileNotFoundError:
        print(f'File {aoi_filename} not found')